In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
import statistics
from IPython.display import HTML, display
from wimbledon.vis.Visualise import DataHandlers

In [2]:
with open('github.token', 'r') as f:
    token = f.read().strip()

headers = {"Authorization": "Bearer " + token}


def run_query(query):  # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.github.com/graphql', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))


GitHub emojis
=======

Project issues that members of the REG team have selected, "LAUGH" are the ones they would most like to be assigned to, "THUMBS_UP" they would also be happy with, "THUMBS_DOWN" they would not be happy with.

In [3]:
emojis = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '👍',
          'LAUGH': '✅'}

In [4]:
fc = DataHandlers.Forecast()  # get data from forecast

Enpoint: whoami | Time: 0.5045343580131885 
Enpoint: people/475627 | Time: 0.48001563697471283 

AUTHENTICATED USER:
Ed Chalstrey echalstrey@turing.ac.uk

CLIENTS
Enpoint: clients | Time: 0.5022566420084331 
PROJECTS
Enpoint: projects | Time: 0.7846791759948246 
ROLES
Enpoint: roles | Time: 0.5929673300124705 
PEOPLE
Enpoint: people | Time: 0.6181879299983848 
PLACEHOLDERS
Enpoint: placeholders | Time: 0.4605285070138052 
MILESTONES
Enpoint: milestones | Time: 0.4705765710095875 
ASSIGNMENTS
Enpoint: assignments | Time: 0.8025056769838557 
DONE! (5.3s)


REG team
=====

We need a list of the team members full names from Forecast to compare with their names on GitHub. They match most of the time except when there is no GitHub name. Mapping below covers the rest of the team. We need to update this list if someone new is added to Forecast who doesn't have a matching GitHub name.

In [5]:
names = list(fc.people.full_name)
names.remove('Giovanni Colavizza')
names.remove('Miguel Morin')
names.remove('Mohammed Ali Al-Badri')
names

['Oliver Strickson',
 'Angus Williams',
 'David Beavan',
 'Evelina Gabasova',
 'James Geddes',
 'James Hetherington',
 'Louise Bowler',
 "Martin O'Reilly",
 'May Yong',
 'Nick Barlow',
 'Radka Jersakova',
 'Timothy Hobson',
 'Tomas Lazauskas',
 'Camila Rangel Smith',
 'Sarah Gibson',
 'James Robinson',
 'Eric Daub',
 'Jim Madge',
 'Amber Raza',
 'Kasra Hosseini',
 'Oscar Giles',
 'Joel Dearden',
 'Amaani Hoddoon',
 'Jack Roberts',
 'Oliver Forrest',
 'Gabriel Hanganu',
 'Ed Chalstrey',
 'Ashwini Venkatasubramaniam',
 'Roly Perera']

In [6]:
gid_mapping = {  # People without their full names on github.
 'myyong': 'May Yong',
 'nbarlowATI': 'Nick Barlow',
 'thobson88': 'Timothy Hobson',
 'miguelmorin': 'Miguel Morin',
 'OscartGiles': 'Oscar Giles',
 'AshwiniKV': 'Ashwini Venkatasubramaniam',
}

Find a team member's availability for a date range
---

In [9]:
def get_person_availability(name, start_date, end_date):
    try:
        person_id = fc.get_person_id(name)
    except:
        return 0.0
    peopledf = 1 - fc.people_totals.resample('MS').mean()  # pandas df for team members availability
    peopledf = peopledf[(peopledf.index >= start_date) & (peopledf.index <= end_date)]
    availability_range = peopledf[person_id]
    average_availability = statistics.mean(availability_range)
    return round(average_availability, 2)

In [10]:
start_date = datetime(2019, 10, 1)
end_date = datetime(2020, 10, 1)
get_person_availability("Ed Chalstrey", start_date, end_date)

0.77

Create queries with GraphQL to get the emojis for each GitHub issue
===

In [11]:
issues = fc.projects["GitHub"].dropna().values  # Get list of GitHub issues for projects

In [12]:
query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    issue(number:X) {
          number
          title
          url

          reactionGroups {
            content
            users(first:20) {
                edges {
                    node {
                        login
                        name
                    }
                }
            }
            }
    }
  }
}
"""

Check if a GitHub issue has emojis, based on the result of GraphQL query
----

In [13]:
def query_result_contains_emojis(query_result):
    for reaction in query_result['data']['repository']['issue']['reactionGroups']:
        if len(reaction['users']['edges']) > 0:
            return True
    return False

Generate a HTML table displaying the results of emoji rating and availability of each team member for each project
---

In [14]:
def collate_emoji_data(data, query_result, names, start_date, end_date, requirement, project_name):
    emojis_availability = [project_name]
    for name in names:                 
        person_availability = get_person_availability(name, start_date, end_date)
#         if person_availability >= 0.1:
        percentage_availability = round((person_availability / requirement) * 100, 2)
        emoji_name = None
        for reaction in query_result['data']['repository']['issue']['reactionGroups']:
            for edge in reaction['users']['edges']:
                if edge['node']['name'] == name:
                    emoji_name = reaction['content']
                    if not emoji_name:
                        if gid_mapping[edge['node']['login']] == name:
                            emoji_name = reaction['content']
        if emoji_name:
            emoji = emojis[emoji_name]
        else:
            emoji = "❓"
        emojis_availability.append(emoji + " " + str(percentage_availability) + "% (" + str(person_availability) + ")")  
    data.append(emojis_availability)
    return data

def tabulate_emoji_data(data):
    display(HTML(
   '<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
       )
    ))

Get the projects for which resources (team members) are required and display availability and preference for each team member who has with reacted with an emoji on the GitHub issue
----

In [15]:
resreqdf = fc.project_resourcereq.resample('MS').mean() # grouped by month and mean taken

In [16]:
names_copy = names.copy()
names_copy.insert(0, "Person")
data = [
        names_copy
    ]
for project_id in resreqdf:
    for resreq in resreqdf[project_id]:
        if resreq > 0:  # if at least one month in the dataframe has a resource requirement of more than 0 FTE
            issue_num = fc.projects.loc[project_id, "GitHub"]
            if issue_num in issues:
                first_resreq_date = resreqdf.index[resreqdf[project_id] != 0][0].strftime("%Y-%m-%d")
                project_title = str(issue_num) + " " + fc.projects.loc[project_id, "name"] + ": " + first_resreq_date + " to " + fc.projects.loc[project_id, "end_date"] + "\n" + str(round(resreq, 3)) + " FTE"
                modified_query = query.replace("X", str(issue_num))
                result = run_query(modified_query)  # Execute the query
                data = collate_emoji_data(data, result, names, first_resreq_date, fc.projects.loc[project_id, "end_date"], resreq, project_title)
            break
tabulate_emoji_data(data)

Person,Oliver Strickson,Angus Williams,David Beavan,Evelina Gabasova,James Geddes,James Hetherington,Louise Bowler,Martin O'Reilly,May Yong,Nick Barlow,Radka Jersakova,Timothy Hobson,Tomas Lazauskas,Camila Rangel Smith,Sarah Gibson,James Robinson,Eric Daub,Jim Madge,Amber Raza,Kasra Hosseini,Oscar Giles,Joel Dearden,Amaani Hoddoon,Jack Roberts,Oliver Forrest,Gabriel Hanganu,Ed Chalstrey,Ashwini Venkatasubramaniam,Roly Perera
96 Living With Machines: 2019-07-01 to 2022-03-31 1.0 FTE,❓ 70.0% (0.7),❓ 0.0% (0.0),❓ 41.0% (0.41),👍 41.0% (0.41),❓ 0.0% (0.0),❓ 10.0% (0.1),👍 67.0% (0.67),❓ 18.0% (0.18),❓ 80.0% (0.8),❓ 77.0% (0.77),👍 89.0% (0.89),❓ 41.0% (0.41),❌ 74.0% (0.74),👍 83.0% (0.83),✅ 70.0% (0.7),👍 70.0% (0.7),❓ 70.0% (0.7),❌ 79.0% (0.79),❓ 100.0% (1.0),✅ 59.0% (0.59),❓ 91.0% (0.91),❓ 0.0% (0.0),❓ 100.0% (1.0),✅ 67.0% (0.67),❓ 100.0% (1.0),❓ 100.0% (1.0),❓ 39.0% (0.39),❓ 41.0% (0.41),❓ 30.0% (0.3)
311 AIDA-Lloyds: 2019-08-01 to 2019-12-31 0.4 FTE,❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),👍 0.0% (0.0),❓ 0.0% (0.0),❓ 25.0% (0.1),👍 100.0% (0.4),❓ 0.0% (0.0),❓ 50.0% (0.2),❓ 0.0% (0.0),❓ 125.0% (0.5),❓ 0.0% (0.0),❌ 50.0% (0.2),✅ 25.0% (0.1),👍 0.0% (0.0),👍 2.5% (0.01),❓ 0.0% (0.0),❓ 125.0% (0.5),❓ 250.0% (1.0),👍 50.0% (0.2),❓ 127.5% (0.51),❓ 0.0% (0.0),❓ 250.0% (1.0),❓ 125.0% (0.5),❓ 250.0% (1.0),❓ 250.0% (1.0),❓ 250.0% (1.0),❓ 150.0% (0.6),❓ 250.0% (1.0)
183 Probabilistic FEM: 2019-07-01 to 2020-02-29 0.5 FTE,❓ 24.0% (0.12),❓ 0.0% (0.0),❓ 24.0% (0.12),❌ 24.0% (0.12),❓ 0.0% (0.0),✅ 20.0% (0.1),❌ 88.0% (0.44),❓ 0.0% (0.0),❓ 50.0% (0.25),❓ 24.0% (0.12),❌ 112.0% (0.56),❓ 24.0% (0.12),❌ 62.0% (0.31),❌ 62.0% (0.31),❌ 24.0% (0.12),❌ 26.0% (0.13),❓ 24.0% (0.12),👍 100.0% (0.5),❓ 200.0% (1.0),❌ 38.0% (0.19),❓ 126.0% (0.63),❓ 0.0% (0.0),❓ 200.0% (1.0),👍 100.0% (0.5),❓ 200.0% (1.0),❓ 200.0% (1.0),❓ 200.0% (1.0),❓ 138.0% (0.69),❓ 200.0% (1.0)
190 Safe Haven: 2019-02-01 to 2021-03-31 1.0 FTE,❓ 42.0% (0.42),❓ 0.0% (0.0),❓ 29.0% (0.29),❓ 29.0% (0.29),❓ 0.0% (0.0),❓ 9.0% (0.09),❓ 40.0% (0.4),❓ 0.0% (0.0),❓ 56.0% (0.56),❓ 52.0% (0.52),❓ 67.0% (0.67),❓ 29.0% (0.29),👍 54.0% (0.54),❓ 60.0% (0.6),❓ 42.0% (0.42),❌ 42.0% (0.42),❓ 42.0% (0.42),❓ 56.0% (0.56),❓ 100.0% (1.0),❌ 31.0% (0.31),❓ 75.0% (0.75),❓ 0.0% (0.0),❓ 100.0% (1.0),❓ 42.0% (0.42),❓ 100.0% (1.0),❓ 100.0% (1.0),❓ 52.0% (0.52),❓ 56.0% (0.56),❓ 42.0% (0.42)
231 UQM^3: 2019-07-01 to 2020-08-31 0.5 FTE,👍 58.0% (0.29),❓ 0.0% (0.0),❓ 58.0% (0.29),❌ 58.0% (0.29),❓ 0.0% (0.0),❓ 20.0% (0.1),👍 92.0% (0.46),❓ 0.0% (0.0),❓ 106.0% (0.53),❓ 92.0% (0.46),❌ 150.0% (0.75),❓ 58.0% (0.29),✅ 78.0% (0.39),👍 122.0% (0.61),❌ 58.0% (0.29),👍 58.0% (0.29),❓ 58.0% (0.29),👍 100.0% (0.5),❓ 200.0% (1.0),👍 58.0% (0.29),❓ 158.0% (0.79),❓ 0.0% (0.0),❓ 200.0% (1.0),❌ 100.0% (0.5),❓ 200.0% (1.0),❓ 200.0% (1.0),❓ 186.0% (0.93),❓ 164.0% (0.82),❓ 142.0% (0.71)
230 Auto DL: 2019-07-01 to 2019-12-31 0.5 FTE,❌ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),👍 0.0% (0.0),❓ 0.0% (0.0),❓ 20.0% (0.1),👍 84.0% (0.42),❓ 0.0% (0.0),❓ 32.0% (0.16),❓ 0.0% (0.0),👍 84.0% (0.42),❓ 0.0% (0.0),✅ 50.0% (0.25),✅ 16.0% (0.08),👍 0.0% (0.0),✅ 2.0% (0.01),❌ 0.0% (0.0),👍 100.0% (0.5),❓ 200.0% (1.0),👍 50.0% (0.25),❓ 102.0% (0.51),❓ 0.0% (0.0),❓ 200.0% (1.0),👍 100.0% (0.5),❓ 200.0% (1.0),❓ 200.0% (1.0),❌ 200.0% (1.0),❓ 116.0% (0.58),❌ 200.0% (1.0)
308 All in One Cancer imaging optimisation: 2019-09-01 to 2021-08-31 1.0 FTE,❓ 67.0% (0.67),❓ 0.0% (0.0),❓ 42.0% (0.42),👍 42.0% (0.42),❓ 0.0% (0.0),❓ 10.0% (0.1),👍 58.0% (0.58),❓ 10.0% (0.1),❓ 81.0% (0.81),❓ 77.0% (0.77),👍 92.0% (0.92),❓ 42.0% (0.42),👍 69.0% (0.69),👍 85.0% (0.85),👍 67.0% (0.67),❌ 67.0% (0.67),❌ 67.0% (0.67),❌ 75.0% (0.75),❓ 100.0% (1.0),👍 48.0% (0.48),❓ 92.0% (0.92),❓ 0.0% (0.0),❓ 100.0% (1.0),👍 58.0% (0.58),❓ 100.0% (1.0),❓ 100.0% (1.0),✅ 46.0% (0.46),❓ 52.0% (0.52),❌ 33.0% (0.33)
317 Fusion modelling- IDEA: 2019-09-01 to 2021-08-31 1.0 FTE,❓ 67.0% (0.67),❓ 0.0% (0.0),❓ 42.0% (0.42),❌ 42.0% (0.42),❓ 0.0% (0.0),❓ 10.0% (0.1),❓ 58.0% (0.58),❓ 10.0% (0.1),❓ 81.0% (0.81),❓ 77.0% (0.77),❓ 92.0% (0.92),❓ 42.0% (0.42),